In [1]:
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn
#!pip install statsmodels
#!pip install -U scikit-learn scipy matplotlib

In [2]:
#Did not remove any feature

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def mean_absolute_error(y, yp):
    mae = sum(abs(y - yp)) / len(y)
    return mae


df = pd.read_csv('SeoulBikeData.csv', encoding='gbk', index_col=['Date'])
data = df.copy()

# find the column names with odd codes and change them
data = data.rename(columns={'Temperature(癈)':'Temperature(C)', 'Dew point temperature(癈)':'Dew point temperature(C)'})

# select only the data entries of ['Functioning Day'] == 'Yes' and drop the 'Functioning Day' feature
data = data[data['Functioning Day'] == 'Yes']
data.pop('Functioning Day')

# convert the text features into numeric
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Seasons'] = le.fit_transform(data['Seasons'])
data['Holiday'] = le.fit_transform(data['Holiday'])

target = data.pop('Rented Bike Count')

mean_performance_train = []
mean_performance_test = []
mean_MAE = []

run_times = 100

for i in range (run_times):
    # start training and testing the multiple regression model
    from sklearn.model_selection import train_test_split
    data_train, data_test, target_train, target_test = train_test_split(data.values, target.values, test_size=0.2)
    
    from sklearn.linear_model import LinearRegression
    reg_model = LinearRegression()
    reg_model.fit(data_train, target_train)
    predictions = reg_model.predict(data_test)
    
    #print ('MAE = ', mean_absolute_error(target_test,predictions).round(3))
    mean_MAE.append(mean_absolute_error(target_test,predictions))
    #print ('Score of Train Set = ', reg_model.score(data_train, target_train).round(3))
    mean_performance_train.append(reg_model.score(data_train, target_train))
    #print ('Score of Test Set = ', reg_model.score(data_test, target_test).round(3))
    mean_performance_test.append(reg_model.score(data_test, target_test))
    #print ()

print( 'Mean MAE = ', np.mean(mean_MAE).round(3))
print( 'Std 0f MAE = ', np.std(mean_MAE).round(3))
print()
print( 'Mean Score of Train Set = ', np.mean(mean_performance_train).round(3))
print( 'Std Score of Train Set = ', np.std(mean_performance_train).round(3))
print()
print( 'Mean Score of Test Set = ', np.mean(mean_performance_test).round(3))
print( 'Std Score of Test Set = ', np.std(mean_performance_test).round(3))

Mean MAE =  325.605
Std 0f MAE =  6.532

Mean Score of Train Set =  0.541
Std Score of Train Set =  0.003

Mean Score of Test Set =  0.54
Std Score of Test Set =  0.012


In [3]:
import time
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels import stats
import scipy
from datetime import datetime
import matplotlib.pyplot as plt

def mean_absolute_error(y, yp):
    mae = sum(abs(y - yp)) / len(y)
    return mae

start_time = time.time()

df = pd.read_csv('SeoulBikeData.csv', encoding='gbk', index_col=['Date'])
data = df.copy()
#print (df.describe())

# find the column names with odd codes and change them
data = data.rename(columns={'Temperature(癈)':'Temperature(C)', 'Dew point temperature(癈)':'Dew point temperature(C)'})

# select only the data entries of ['Functioning Day'] == 'Yes' and drop the 'Functioning Day' feature
data = data[data['Functioning Day'] == 'Yes']
data.pop('Functioning Day')

# convert the text features into numeric
from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder

le = LabelEncoder()
data['Seasons'] = le.fit_transform(data['Seasons'])
data['Holiday'] = le.fit_transform(data['Holiday'])

max_performance_test = 0
order = 0

#data.pop('Hour')
#data.pop('Temperature(C)')
#data.pop('Humidity(%)')
#data.pop('Wind speed (m/s)')
#data.pop('Visibility (10m)')
#data.pop('Dew point temperature(C)')
#data.pop('Solar Radiation (MJ/m2)')
#data.pop('Rainfall(mm)')
#data.pop('Snowfall (cm)')
#data.pop('Seasons')
#data.pop('Holiday')

#print (np.mean(data['Rented Bike Count']))
#print (np.std(data['Rented Bike Count']))
#mu = np.mean(data['Rented Bike Count'])
#sigma = np.std(data['Rented Bike Count'])
#data = data[ (mu-2*sigma<= data['Rented Bike Count']) & (data['Rented Bike Count']<= mu+2*sigma)]

target = data.pop('Rented Bike Count')

# https://reurl.cc/mMoMWM
# Statistic
data = sm.add_constant(data)
Statistic = sm.OLS(target,data).fit()
#print(Statistic.summary())
#print()

# Residual
res = target - Statistic.fittedvalues
#print(res)

#建立線性回歸時，必須要檢驗殘差(Residaul)是否符合以下三個假設：
#	• 常態性 ( Normality )
#	• 獨立性 ( Independence )
#	• 變異數同質性 ( Homogeneity of Variance )

# Normalty 
# Not Normal distribution, if p-value < 0.05  
print (scipy.stats.shapiro(res))
if (scipy.stats.shapiro(res).pvalue < 0.05) : print ('The Residual is not normally distributed')
else : print ('The Residual is normally distributed')
print()

# Sample Inependence
# 每個殘差之間互為獨立：可使用 Durban-Watson test 檢定每個殘差之間是否獨立。
# p-value > 0.05：接受虛無假設，代表每個殘差之間互為獨立。
tstatistic = stats.stattools.durbin_watson(res) # 計算出 T 統計量
tpvalue = scipy.stats.t.sf(tstatistic, 504) # 將統計量轉換成 p-value
print('t-statistic =', tstatistic, '\n', 'P-Value =', tpvalue)
print()

# Variance Homogeneity
# 殘差變異數一致性：使用 Breusch-Pagan Test 檢測殘差變異數是否一致
#  p-value < 0.05：拒絕虛無假設，代表殘差變異數不一致。
print(sms.het_breuschpagan(res, Statistic.model.exog))
print()
    

mean_performance_train = []
mean_performance_test = []
mean_MAE = []

run_times = 100

for i in range (run_times):
    # start training and testing the multiple regression model
    from sklearn.model_selection import train_test_split
    data_train, data_test, target_train, target_test = train_test_split(data.values, target.values, test_size=0.2)
    
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import make_pipeline
    
    #reg_model = LinearRegression()
    
    #reg_model.fit(data_train, target_train)
    poly_model = make_pipeline(PolynomialFeatures(2), LinearRegression())
    poly_model.fit(data_train, target_train)
    
    #predictions = reg_model.predict(data_test)
    predictions = poly_model.predict(data_test)

    mean_MAE.append(mean_absolute_error(target_test,predictions))
    mean_performance_train.append(poly_model.score(data_train, target_train))
    mean_performance_test.append(poly_model.score(data_test, target_test))

print( 'Mean MAE = ', np.mean(mean_MAE).round(3))
print( 'Std 0f MAE = ', np.std(mean_MAE).round(3))
print()
print( 'Mean Score of Train Set = ', np.mean(mean_performance_train).round(3))
print( 'Std Score of Train Set = ', np.std(mean_performance_train).round(3))
print()
print( 'Mean Score of Test Set = ', np.mean(mean_performance_test).round(3))
print( 'Std Score of Test Set = ', np.std(mean_performance_test).round(3))
print()

print('Execution time :', round((time.time()-start_time),3), 'seconds')

C:\Users\user\AppData\Roaming\Python\Python312\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 8465.
  res = hypotest_fun_out(*samples, **kwds)


ShapiroResult(statistic=0.9510241718694438, pvalue=1.8696488055007193e-46)
The Residual is not normally distributed

t-statistic = 0.520178830724128 
 P-Value = 0.3015836737814327

(777.5917372986112, 1.2247093706776786e-159, 77.73021603838109, 1.1791218105864267e-167)

Mean MAE =  270.472
Std 0f MAE =  5.215

Mean Score of Train Set =  0.666
Std Score of Train Set =  0.003

Mean Score of Test Set =  0.656
Std Score of Test Set =  0.012

Execution time : 3.169 seconds
